In [1]:
from GG_mthesis import *
import metpy
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.units import units
%matplotlib inline

sns.set_style('whitegrid')
sns.set_palette('colorblind')
sns.set(font_scale=1.2)

In [2]:
os.getcwd()

'C:\\Users\\gian_\\repos\\mthesis_gianlucca'

In [3]:
#Start year and end year
START_YEAR  = 2017
END_YEAR    = 2019 #NOT INCLUDED
NUMBER_YEAR = 2


years = range(START_YEAR, END_YEAR)
months = ['%.2d' % i for i in range(1,13)]
number_years = range(NUMBER_YEAR)

#Input paths of the files
base_adt_path = 'data/01_raw/AVISO/SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047/dataset-duacs-rep-global-merged-allsat-phy-l4' #
#adt_file_name_base = 'dt_global_twosat_phy_l4_'


#Main file reading algorithm. Multifile reader of xarray which concanates the read files (loading through dask). Additional loop and concanating of each month and years (THIS IS JUST FOR THE FIRST YEAR, CHANGE INDEX OF YEARS (MAY HAVE TO LOOP THROUGH))
ds_adt = xr.concat([xr.concat([[xr.open_mfdataset(base_adt_path + '/' + str(i_year) + '/' + i_month + '/' + '*.nc', parallel = True) for i_month in months] for i_year in years][n_year], dim='time') for n_year in number_years], dim='time')

#Center longitude format to Atlantic.
ds_adt = ds_adt.assign_coords(longitude=(((ds_adt.longitude + 180) % 360)-180)).sortby('longitude')
ds_adt.longitude.attrs = {
    'long_name': 'Longitude',
    'standard_name': 'longitude',
    'units': 'degrees_east',
    'bounds': 'lon_bnds',
    'axis': 'X',
    'valid_max': 179.9375,
    'valid_min': -179.9375,
    '_CoordinateAxisType': 'Lon'}

C:\Users\gian_\miniconda3\envs\mthesis\lib\site-packages\xarray\core\indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
C:\Users\gian_\miniconda3\envs\mthesis\lib\site-packages\xarray\core\indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
C:\Users\gia

In [4]:
# # Comparison between ADT calculated by AVISO and calculated independently:

# #Start year and end year
# START_YEAR  = 2017
# END_YEAR    = 2018 #NOT INCLUDED

# years = range(START_YEAR, END_YEAR)
# months = ['%.2d' % i for i in range(1,13)]
# #Input paths of the files
# base_adt_path = 'data/AVISO/dataset-duacs-rep-global-merged-twosat-phy-l4/' #
# adt_file_name_base = 'dt_global_twosat_phy_l4_'


# #Main file reading algorithm. Multifile reader of xarray which concanates the read files (loading through dask). Additional loop and concanating of each month and years (THIS IS JUST FOR THE FIRST YEAR, CHANGE INDEX OF YEARS (MAY HAVE TO LOOP THROUGH))
# ds_adt = xr.concat([[xr.open_mfdataset(
#     base_adt_path + str(i_year) + '/'
#      + i_month + '/' + '*.nc', 
#      parallel = True) 
#      for i_month in months] for i_year in years][0], 
#      dim='time')

# #Center longitude format to Atlantic.
# ds_adt = ds_adt.assign_coords(longitude=(((ds_adt.longitude + 180) % 360)-180)).sortby('longitude')
# ds_adt.longitude.attrs = {
#     'long_name': 'Longitude',
#     'standard_name': 'longitude',
#     'units': 'degrees_east',
#     'bounds': 'lon_bnds',
#     'axis': 'X',
#     'valid_max': 179.9375,
#     'valid_min': -179.9375,
#     '_CoordinateAxisType': 'Lon'}

In [5]:
#MDT

#Input paths of files 
base_mdt_path   = 'data/01_raw/AVISO/'
mdt_file_name   = 'cnes_obs-sl_glo_phy-mdt_my_0.125deg_P20Y_1625495999270.nc'

#Load the date.
ds_mdt = xr.load_dataset(base_mdt_path + mdt_file_name)

#Interpolate to sla raster.
ds_mdt = ds_mdt.interp(longitude = ds_adt.longitude, latitude = ds_adt.latitude)

In [6]:
ds_adt['adt_calc'] = ((ds_mdt.mdt.isel(time=0) + ds_adt.sla).transpose('time','latitude','longitude'))

ds_adt['adt_calc'].attrs = {'comment': 'The absolute dynamic topography is the sea surface height above geoid; the adt is obtained as follows: adt=sla+mdt where mdt is the mean dynamic topography; calculated here',
 'grid_mapping': 'crs',
 'long_name': 'Absolute dynamic topography',
 'standard_name': 'sea_surface_height_above_geoid',
 'units': 'm'}

In [7]:
#select SO and quantify with metpy
ds_adt_SO = ds_adt.sel(latitude=slice(-80,-40)).metpy.quantify()

In [8]:
#Calculate the ADT meridional gradient in m/km
ds_adt_SO['adt_grad_mer'] = (mpcalc.gradient(ds_adt_SO.adt, axes=['latitude']))[0]*units('m/m').to('m/km') #creates a tuple, we have to select the first element to get the DataArray (that's why the [0])

#Calculate the ADT meridional gradient in m/km (calculated ADT)
ds_adt_SO['adt_grad_mer_calc'] = (mpcalc.gradient(ds_adt_SO.adt_calc, axes=['latitude']))[0]*units('m/m').to('m/km') #creates a tuple, we have to select the first element to get the DataArray (that's why the [0])

#Calculate the ADT radial gradient in m/km
ds_adt_SO['adt_grad_radial'] = (mpcalc.gradient(ds_adt_SO.adt, axes=['longitude','latitude']))[0]*units('m/m').to('m/km')

In [9]:
ds_adt_SO = ds_adt_SO.metpy.dequantify()
ds_adt_SO['adt_grad_mer'].attrs = {
    'long name':'Meridional Gradient of Absolute Dynamic Topographic',
    'description':'Meridional Gradient of the Absolute Dynamic Topography (AVISO), calculated using the metpy gradient function.',
    'units':'meter / kilometer'
}

ds_adt_SO['adt_grad_mer_calc'].attrs = {
    'long name':'Meridional Gradient of Absolute Dynamic Topographic (calculated)',
    'description':'Meridional Gradient of the Absolute Dynamic Topography (calculated using MDT and SLA from AVISO), calculated using the metpy gradient function.',
    'units':'meter / kilometer'
}

ds_adt_SO['adt_grad_radial'].attrs = {
    'long name':'Radial Gradient of Absolute Dynamic Topographic (calculated)',
    'description':'Radial Gradient of the Absolute Dynamic Topography (AVISO), calculated using the metpy gradient function.',
    'units':'meter / kilometer'
}

In [13]:
len(ds_adt_SO.time)

730

In [14]:
#Let's save this dataset in order to load it and not having to repeat all the calculation steps every time (high computing time.)
#The file is too large to be saved in a single file, so I create daily output files which can be loaded using xr.open_mfdataset()
path_to_base = r'./data/04_models/geo_fronts/adt_gradient/'
for i_time in range(len(ds_adt_SO.time)):
        ds_adt_SO.isel(time=i_time).expand_dims(dim='time').to_netcdf(path_to_base + 'ADT_gradients_SO_day_' + str(i_time+1) + '.nc', format='NETCDF4')
        print('Saving day ' + str(i_time + 1))

Saving day 1
Saving day 2
Saving day 3
Saving day 4
Saving day 5
Saving day 6
Saving day 7
Saving day 8
Saving day 9
Saving day 10
Saving day 11
Saving day 12
Saving day 13
Saving day 14
Saving day 15
Saving day 16
Saving day 17
Saving day 18
Saving day 19
Saving day 20
Saving day 21
Saving day 22
Saving day 23
Saving day 24
Saving day 25
Saving day 26
Saving day 27
Saving day 28
Saving day 29
Saving day 30
Saving day 31
Saving day 32
Saving day 33
Saving day 34
Saving day 35
Saving day 36
Saving day 37
Saving day 38
Saving day 39
Saving day 40
Saving day 41
Saving day 42
Saving day 43
Saving day 44
Saving day 45
Saving day 46
Saving day 47
Saving day 48
Saving day 49
Saving day 50
Saving day 51
Saving day 52
Saving day 53
Saving day 54
Saving day 55
Saving day 56
Saving day 57
Saving day 58
Saving day 59
Saving day 60
Saving day 61
Saving day 62
Saving day 63
Saving day 64
Saving day 65
Saving day 66
Saving day 67
Saving day 68
Saving day 69
Saving day 70
Saving day 71
Saving day 72
S